In this notebook, we'll simulate a preferential attachment network using a graph and investigate its betti numbers. We'll first start by building a preferential attachment graph

In [ ]:
import numpy as np
import igraph as ig
import tqdm
from numba import jit
from numba.typed import List

In [ ]:
# upper bound
def betti2_upper_bound(graph):

    edge_list = np.array([e.tuple for e in graph.es])
    start = 0
    num_edges_per_new_node = int(len(edge_list)/(edge_list[-1][1]))
    end = num_edges_per_new_node
    new_betti1_in_batch = [0]
    new_betti2_in_batch = [0]

    while start < len(edge_list) and end <= len(edge_list):
        # find parents of each node
        parents_list = edge_list[start:end, 0]
        parents_list = np.unique(parents_list)

        # generate subgraph for the parents
        subgraph = graph.induced_subgraph(parents_list)

        # count the betti numbers
        mat = get_age_matrix(subgraph)
        dgms = ripser(mat, distance_matrix=True, maxdim=2)['dgms']

        betti1 = 0
        for k, j in dgms[1]:
            betti1 += 1
            if j < float('inf'):
                betti1 -= 1

        betti2 = 0
        for k, j in dgms[2]:
            betti2 += 1
            if j < float('inf'):
                betti2 -= 1

        new_betti1_in_batch.append(betti1)
        new_betti2_in_batch.append(betti2)
        start += num_edges_per_new_node
        end += num_edges_per_new_node
    return np.cumsum(new_betti1_in_batch), np.cumsum(new_betti2_in_batch)